Data base connection

In [8]:
# Librerias
import yaml
import psycopg2 
import sqlalchemy
from psycopg2 import sql
from sqlalchemy import create_engine, text
from sqlalchemy.types import Integer, String, DateTime, Float
import pandas as pd
import os
import re
import unidecode

In [9]:
def load_config(file_path="config.yaml"):
    with open(file_path, "r") as file: # Abre el archivo en modo lectura ("r")
        return yaml.safe_load(file) # Carga y devuelve el contenido del archivo YAML

In [10]:
config = load_config()
db_config = config["database"]

# Load credentials, se accede a los valores de la clave "database" del archivo config.yaml
db_user = db_config["user"]
db_password = db_config["password"]
db_host = db_config["host"]
db_port = db_config["port"]
db_name = db_config["name"]

# DB connection
conn = psycopg2.connect(
    dbname="postgres",
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)
conn.autocommit = True

In [11]:
#Creating the database 
db_name = "ETL_Project"
try:
    with conn.cursor() as cur:
        cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
        print(f"Base de datos '{db_name}' creada exitosamente.")
except psycopg2.errors.DuplicateDatabase:
    print(f"La base de datos '{db_name}' ya existe.")
finally:
    conn.close() #Close the database connection

La base de datos 'ETL_Project' ya existe.


File Reading for Review and Table Creation

In [12]:
matricula_path = "D:/JENN/Maestria IACD/ETL/ETL-Class-Project/MEN_MATRICULA_ESTADISTICA_ES_20250222.csv"
# Leer el archivo en un dataframe
matricula_df = pd.read_csv(matricula_path, sep=',')
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
matricula_df.head()    

C:\Users\jenni\AppData\Local\Temp\ipykernel_23240\2787825400.py:3: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  matricula_df = pd.read_csv(matricula_path, sep=',')


,Código de la Institución,IES PADRE,Institución de Educación Superior (IES),Principal oSeccional,Id_Sector,Id_Caracter,Código del departamento(IES),Departamento de domicilio de la IES,Código del Municipio(IES),Municipio dedomicilio de la IES,Código SNIES delprograma,Programa Académico,Id_Nivel,Id_Nivel_Formacion,Id_Metodologia,Id_Area,Id_Nucleo,Núcleo Básico del Conocimiento (NBC),Código del Departamento(Programa),Departamento de oferta del programa,Código del Municipio(Programa),Municipio de oferta del programa,Id Género,Año,Semestre,Total Matriculados
0,2833,2833,CORPORACION UNIVERSITARIA REMINGTON,PRINCIPAL,2,3,5,ANTIOQUIA,5001,MEDELLIN,13061,CONTADURIA PUBLICA,1,6,2,6,612,CONTADURIA PUBLICA,23,CORDOBA,23670,SAN ANDRES DE SOTAVENTO,2,2015,1,2
1,2833,2833,CORPORACION UNIVERSITARIA REMINGTON,PRINCIPAL,2,3,5,ANTIOQUIA,5001,MEDELLIN,13061,CONTADURIA PUBLICA,1,6,2,6,612,CONTADURIA PUBLICA,23,CORDOBA,23670,SAN ANDRES DE SOTAVENTO,1,2015,1,2
2,2833,2833,CORPORACION UNIVERSITARIA REMINGTON,PRINCIPAL,2,3,5,ANTIOQUIA,5001,MEDELLIN,13061,CONTADURIA PUBLICA,1,6,2,6,612,CONTADURIA PUBLICA,23,CORDOBA,23672,SAN ANTERO,2,2015,1,9
3,2833,2833,CORPORACION UNIVERSITARIA REMINGTON,PRINCIPAL,2,3,5,ANTIOQUIA,5001,MEDELLIN,13061,CONTADURIA PUBLICA,1,6,2,6,612,CONTADURIA PUBLICA,23,CORDOBA,23672,SAN ANTERO,1,2015,1,1
4,2833,2833,CORPORACION UNIVERSITARIA REMINGTON,PRINCIPAL,2,3,5,ANTIOQUIA,5001,MEDELLIN,13061,CONTADURIA PUBLICA,1,6,2,6,612,CONTADURIA PUBLICA,23,CORDOBA,23675,SAN BERNARDO DEL VIENTO,2,2015,1,18


In [13]:
# Reconnect to the ETL_Project database
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
conn = engine.connect()

# Define the table schema
metadata = sqlalchemy.MetaData()

matricula_table = sqlalchemy.Table('Table_Matricula', metadata,
    sqlalchemy.Column('codigo_institucion', sqlalchemy.Integer),
    sqlalchemy.Column('ies_padre', sqlalchemy.Integer),
    sqlalchemy.Column('institucion_educacion_superior', sqlalchemy.String),
    sqlalchemy.Column('principal_seccional', sqlalchemy.String),
    sqlalchemy.Column('id_sector', sqlalchemy.Integer),
    sqlalchemy.Column('id_caracter', sqlalchemy.Integer),
    sqlalchemy.Column('codigo_departamento_ies', sqlalchemy.Integer),
    sqlalchemy.Column('departamento_domicilio_ies', sqlalchemy.String),
    sqlalchemy.Column('codigo_municipio_ies', sqlalchemy.Integer),
    sqlalchemy.Column('municipio_domicilio_ies', sqlalchemy.String),
    sqlalchemy.Column('codigo_snies_programa', sqlalchemy.Integer),
    sqlalchemy.Column('programa_academico', sqlalchemy.String),
    sqlalchemy.Column('id_nivel', sqlalchemy.Integer),
    sqlalchemy.Column('id_nivel_formacion', sqlalchemy.Integer),
    sqlalchemy.Column('id_metodologia', sqlalchemy.Integer),
    sqlalchemy.Column('id_area', sqlalchemy.String),
    sqlalchemy.Column('id_nucleo', sqlalchemy.String),
    sqlalchemy.Column('nucleo_basico_conocimiento', sqlalchemy.String),
    sqlalchemy.Column('codigo_departamento_programa', sqlalchemy.Integer),
    sqlalchemy.Column('departamento_oferta_programa', sqlalchemy.String),
    sqlalchemy.Column('codigo_municipio_programa', sqlalchemy.Integer),
    sqlalchemy.Column('municipio_oferta_programa', sqlalchemy.String),
    sqlalchemy.Column('id_genero', sqlalchemy.Integer),
    sqlalchemy.Column('ano', sqlalchemy.Integer),
    sqlalchemy.Column('semestre', sqlalchemy.Integer),
    sqlalchemy.Column('total_matriculados', sqlalchemy.Integer)
)

# Create the table in the database
metadata.create_all(engine)

# Insert the data from the DataFrame into the table
matricula_df.to_sql('Table_Matricula', engine, if_exists='replace', index=False)
print("Data inserted successfully.")

Data inserted successfully.


In [14]:
with engine.connect() as conn:
    table_matricula_df = pd.read_sql('SELECT * FROM "Table_Matricula"', conn)

In [15]:
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
table_matricula_df.head()

,Código de la Institución,IES PADRE,Institución de Educación Superior (IES),Principal oSeccional,Id_Sector,Id_Caracter,Código del departamento(IES),Departamento de domicilio de la IES,Código del Municipio(IES),Municipio dedomicilio de la IES,Código SNIES delprograma,Programa Académico,Id_Nivel,Id_Nivel_Formacion,Id_Metodologia,Id_Area,Id_Nucleo,Núcleo Básico del Conocimiento (NBC),Código del Departamento(Programa),Departamento de oferta del programa,Código del Municipio(Programa),Municipio de oferta del programa,Id Género,Año,Semestre,Total Matriculados
0,2833,2833,CORPORACION UNIVERSITARIA REMINGTON,PRINCIPAL,2,3,5,ANTIOQUIA,5001,MEDELLIN,13061,CONTADURIA PUBLICA,1,6,2,6,612,CONTADURIA PUBLICA,23,CORDOBA,23670,SAN ANDRES DE SOTAVENTO,2,2015,1,2
1,2833,2833,CORPORACION UNIVERSITARIA REMINGTON,PRINCIPAL,2,3,5,ANTIOQUIA,5001,MEDELLIN,13061,CONTADURIA PUBLICA,1,6,2,6,612,CONTADURIA PUBLICA,23,CORDOBA,23670,SAN ANDRES DE SOTAVENTO,1,2015,1,2
2,2833,2833,CORPORACION UNIVERSITARIA REMINGTON,PRINCIPAL,2,3,5,ANTIOQUIA,5001,MEDELLIN,13061,CONTADURIA PUBLICA,1,6,2,6,612,CONTADURIA PUBLICA,23,CORDOBA,23672,SAN ANTERO,2,2015,1,9
3,2833,2833,CORPORACION UNIVERSITARIA REMINGTON,PRINCIPAL,2,3,5,ANTIOQUIA,5001,MEDELLIN,13061,CONTADURIA PUBLICA,1,6,2,6,612,CONTADURIA PUBLICA,23,CORDOBA,23672,SAN ANTERO,1,2015,1,1
4,2833,2833,CORPORACION UNIVERSITARIA REMINGTON,PRINCIPAL,2,3,5,ANTIOQUIA,5001,MEDELLIN,13061,CONTADURIA PUBLICA,1,6,2,6,612,CONTADURIA PUBLICA,23,CORDOBA,23675,SAN BERNARDO DEL VIENTO,2,2015,1,18


In [16]:
# Función para quitar tildes y poner en mayúscula
def clean_column_name(col_name):
    return unidecode.unidecode(col_name).upper()

# Aplicar la función a los nombres de las columnas
matricula_df.columns = [clean_column_name(col) for col in matricula_df.columns]

# Mostrar los primeros registros para verificar los cambios
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
matricula_df.head()

,CODIGO DE LA INSTITUCION,IES PADRE,INSTITUCION DE EDUCACION SUPERIOR (IES),PRINCIPAL OSECCIONAL,ID_SECTOR,ID_CARACTER,CODIGO DEL DEPARTAMENTO(IES),DEPARTAMENTO DE DOMICILIO DE LA IES,CODIGO DEL MUNICIPIO(IES),MUNICIPIO DEDOMICILIO DE LA IES,CODIGO SNIES DELPROGRAMA,PROGRAMA ACADEMICO,ID_NIVEL,ID_NIVEL_FORMACION,ID_METODOLOGIA,ID_AREA,ID_NUCLEO,NUCLEO BASICO DEL CONOCIMIENTO (NBC),CODIGO DEL DEPARTAMENTO(PROGRAMA),DEPARTAMENTO DE OFERTA DEL PROGRAMA,CODIGO DEL MUNICIPIO(PROGRAMA),MUNICIPIO DE OFERTA DEL PROGRAMA,ID GENERO,ANO,SEMESTRE,TOTAL MATRICULADOS
0,2833,2833,CORPORACION UNIVERSITARIA REMINGTON,PRINCIPAL,2,3,5,ANTIOQUIA,5001,MEDELLIN,13061,CONTADURIA PUBLICA,1,6,2,6,612,CONTADURIA PUBLICA,23,CORDOBA,23670,SAN ANDRES DE SOTAVENTO,2,2015,1,2
1,2833,2833,CORPORACION UNIVERSITARIA REMINGTON,PRINCIPAL,2,3,5,ANTIOQUIA,5001,MEDELLIN,13061,CONTADURIA PUBLICA,1,6,2,6,612,CONTADURIA PUBLICA,23,CORDOBA,23670,SAN ANDRES DE SOTAVENTO,1,2015,1,2
2,2833,2833,CORPORACION UNIVERSITARIA REMINGTON,PRINCIPAL,2,3,5,ANTIOQUIA,5001,MEDELLIN,13061,CONTADURIA PUBLICA,1,6,2,6,612,CONTADURIA PUBLICA,23,CORDOBA,23672,SAN ANTERO,2,2015,1,9
3,2833,2833,CORPORACION UNIVERSITARIA REMINGTON,PRINCIPAL,2,3,5,ANTIOQUIA,5001,MEDELLIN,13061,CONTADURIA PUBLICA,1,6,2,6,612,CONTADURIA PUBLICA,23,CORDOBA,23672,SAN ANTERO,1,2015,1,1
4,2833,2833,CORPORACION UNIVERSITARIA REMINGTON,PRINCIPAL,2,3,5,ANTIOQUIA,5001,MEDELLIN,13061,CONTADURIA PUBLICA,1,6,2,6,612,CONTADURIA PUBLICA,23,CORDOBA,23675,SAN BERNARDO DEL VIENTO,2,2015,1,18
